# Case Study Census data

Create an Engine and Meta Object
preparing SQLAlchemy and the DataBase
populating the database (import csv)

insert values list into a census table

answering data science questions with queries

determine average age for males and females
determine the percentage of females for each state
determine top 10 states by population change from 2000 to 2008


In [7]:
#create an Engine and MetaData object
from sqlalchemy import create_engine, MetaData

metadata = MetaData()
engine = create_engine('sqlite:///my_census.sqlite')

In [8]:
#quick check of the file we are going to load
#read csv file
import numpy as np
import pandas as pd

census_check = pd.read_csv('census.csv')
census_check.head(3)

,state,sex,age,pop2000,pop2008
0,Illinois,M,0,89600,95012
1,Illinois,M,1,88445,91829
2,Illinois,M,2,88729,89547


In [9]:
#preparing SQLAlchemy and the DataBase
from sqlalchemy import (Table, Column, String, Integer)

census = Table('census', metadata,  #create a table called census
               Column('state', String(30)),  #with these columns
               Column('sex', String(1)),
               Column('age', Integer()),
               Column('pop2000', Integer()),
               Column('pop2008', Integer()))
metadata.create_all(engine)


In [10]:
#quick check

#check created table
engine.table_names()

['census']

In [11]:
#populating the database
#load a csv fle into a values list
import csv
values_list = []
with open('census.csv', 'rt') as f:
    csv_reader = csv.reader(f)
    # Enumerate the rows of csv_reader
    #Within the for loop:
    #Complete the data dictionary by filling in the values for each of the keys. 
    #The values are contained in row. row[0] represents the value for 'state', 
    #row[1] represents the value for 'sex', and so on.
    #Append data to values_list.
    for idx, row in enumerate(csv_reader):
        #create data and append to values_list
        data = {'state': row[0], 'sex': row[1], 'age': row[2], 'pop2000': row[3],'pop2008': row[4]}
        values_list.append(data)
        


In [13]:
#insert values list into a census table
from sqlalchemy import insert
# Create a insert statement for census: stmt
stmt = insert(census)

connection =engine.connect()
result_proxy = connection.execute(stmt, values_list)

print(result_proxy.rowcount)

8773


In [15]:
#quering the database
#answering data science questions with queries
#determine average age for males and females
from sqlalchemy import select,func
stmt = select([census.columns.sex,
              (func.sum(census.columns.pop2008 *
                       census.columns.age) /
              func.sum(census.columns.pop2008)
              ).label('average_age')])

stmt = stmt.group_by(census.columns.sex)

results = connection.execute(stmt).fetchall()

# Print the average age by sex
for result in results:
    print(result.sex, result.average_age)

F 38
M 35
sex None


In [17]:
print(repr(census))

Table('census', MetaData(bind=None), Column('state', String(length=30), table=<census>), Column('sex', String(length=1), table=<census>), Column('age', Integer(), table=<census>), Column('pop2000', Integer(), table=<census>), Column('pop2008', Integer(), table=<census>), schema=None)


In [19]:
#determine the percentage of females for each state
'''
Build a Query to Determine the Percentage of Population by Gender and State
In this exercise, you will write a query to determine the percentage of the population
in 2000 that comprised of women. You will group this query by state.
INSTRUCTIONS

Import case, cast and Float from sqlalchemy.
Define a statement to select state and the percentage of females in 2000.
Inside func.sum(), use case() to select females (using the sex column) from pop2000. 
Remember to specify else_=0 if the sex is not 'F'.
To get the percentage, divide the number of females in the year 2000 by the overall 
population in 2000. Cast the divisor - census.columns.pop2000 - to Float before
multiplying by 100.
Group the query by state.
Execute the query and store it as results.
Print state and percent_female for each record. This has been done for you, so hit 'Submit Answer' to see the result.
'''
from sqlalchemy import case,cast,Float
# Build a query to calculate the percentage of females in 2000: stmt
stmt = select([census.columns.state,
    (func.sum(
    case([
        (census.columns.sex == 'F',census.columns.pop2000)
    ], else_ =0)) /
    cast(func.sum(census.columns.pop2000), Float) * 100).label('percent_female')])

#group query by state
stmt = stmt.group_by(census.columns.state)
#execute the query
results = connection.execute(stmt).fetchall()

print(results)
# Print the average age by sex
#for result in results:
#    print(result.state, result.percent_female)

[('Alabama', 51.832407770179465), ('Alaska', 49.301497893484594), ('Arizona', 50.22361303057914), ('Arkansas', 51.26992846221834), ('California', 50.35233214901979), ('Colorado', 49.84767060299562), ('Connecticut', 51.66816507130644), ('Delaware', 51.61109733558627), ('District of Columbia', 53.129626141738385), ('Florida', 51.36488001165242), ('Georgia', 51.11408350339436), ('Hawaii', 51.118011836915514), ('Idaho', 49.98972623903102), ('Illinois', 51.11224234802867), ('Indiana', 50.95480313297678), ('Iowa', 50.950398342534264), ('Kansas', 50.821864107754735), ('Kentucky', 51.32687036927168), ('Louisiana', 51.75351596554121), ('Maine', 51.50570813418951), ('Maryland', 51.93575549972231), ('Massachusetts', 51.843023571316785), ('Michigan', 50.97246518318712), ('Minnesota', 50.49332944301148), ('Mississippi', 51.92229481794672), ('Missouri', 51.46888602639692), ('Montana', 50.32202690728538), ('Nebraska', 50.8584549336086), ('Nevada', 49.36736361384359), ('New Hampshire', 50.858019844961

In [20]:
# Print the percentage, tidy results
for result in results:
    print(result.state, result.percent_female)

Alabama 51.832407770179465
Alaska 49.301497893484594
Arizona 50.22361303057914
Arkansas 51.26992846221834
California 50.35233214901979
Colorado 49.84767060299562
Connecticut 51.66816507130644
Delaware 51.61109733558627
District of Columbia 53.129626141738385
Florida 51.36488001165242
Georgia 51.11408350339436
Hawaii 51.118011836915514
Idaho 49.98972623903102
Illinois 51.11224234802867
Indiana 50.95480313297678
Iowa 50.950398342534264
Kansas 50.821864107754735
Kentucky 51.32687036927168
Louisiana 51.75351596554121
Maine 51.50570813418951
Maryland 51.93575549972231
Massachusetts 51.843023571316785
Michigan 50.97246518318712
Minnesota 50.49332944301148
Mississippi 51.92229481794672
Missouri 51.46888602639692
Montana 50.32202690728538
Nebraska 50.8584549336086
Nevada 49.36736361384359
New Hampshire 50.858019844961746
New Jersey 51.51713956125773
New Mexico 51.0471720798335
New York 51.83453865150073
North Carolina 51.482262322084594
North Dakota 50.50069363231332
Ohio 51.46550350015544
Okl

In [22]:
#determine top 5 states by population change from 2000 to 2008
'''
Build a Query to Determine the Difference by State from the 2000 and 2008 Censuses
In this final exercise, you will write a query to calculate the states that changed the most
in population. You will limit your query to display only the top 10 states.

Build a statement to:
Select state.
Calculate the difference in population between 2008 (pop2008) and 2000 (pop2000).
Group the query by census.columns.state using the .group_by() method on stmt.
Order by 'pop_change' in descending order using the .order_by() method with the desc()
function on 'pop_change'.
Limit the query to the top 10 states using the .limit() method.
Execute the query and store it as results.
Print the state and the population change for each result. This has been done for you,
so hit 'Submit Answer' to see the result!
'''

from sqlalchemy import desc #to sort out data in descending order

stmt = select([census.columns.state,
              (census.columns.pop2008 -
              census.columns.pop2000).label('pop_change')
              ])

# Group by State
stmt = stmt.group_by(census.columns.state)

stmt = stmt.order_by(desc('pop_change')) # in descending order
#limit top 10
stmt = stmt.limit(10)

# Use connection to execute the statement and fetch all results
results = connection.execute(stmt).fetchall()

# Print the state and population change for each record
for result in results:
    print('{}:{}'.format(result.state, result.pop_change))

Texas:40137
California:35406
Florida:21954
Arizona:14377
Georgia:13357
North Carolina:11574
Virginia:6639
Colorado:6425
Utah:5934
Illinois:5412
